<a href="https://colab.research.google.com/github/22ananya/MUSI6201/blob/main/Final%20Project%20-%20Stereo%20Demixing/FINAL_Run_HAAQI_Eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import generally required packages - update as needed

In [1]:
# Import dependencies
import numpy as np
import matplotlib.pyplot as plt
import librosa
import scipy.signal as sp
import scipy.io.wavfile as wav

Import/Install the prerequisite code for implementing the Cadenza challenge - includes baselines, other important files

In [2]:
!pip install pyclarity==0.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 98.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 53.8 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: file

Import the dataset for the Cadenza challenge directly through the Google Drive link - Only needs to be done once! So now commented out


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Check current path

In [4]:
import os
print(os.getcwd())

/content


Change current path to Cadenza folder

In [5]:
os.chdir('/content/drive/MyDrive/Cadenza_Challenge/cad_icassp_2024/Validation')
print(os.getcwd())

/content/drive/MyDrive/Cadenza_Challenge/cad_icassp_2024/Validation


In [6]:
from __future__ import annotations

import json
import logging
from pathlib import Path

# pylint: disable=import-error
import hydra
import numpy as np
import torch
from numpy import ndarray
from omegaconf import DictConfig
from torchaudio.pipelines import HDEMUCS_HIGH_MUSDB

from clarity.enhancer.compressor import Compressor
from clarity.enhancer.nalr import NALR
from clarity.evaluator.haaqi import compute_haaqi
from clarity.utils.audiogram import Audiogram, Listener
from clarity.utils.file_io import read_signal
from clarity.utils.flac_encoder import FlacEncoder
from clarity.utils.results_support import ResultsFile
from clarity.utils.signal_processing import (
    clip_signal,
    denormalize_signals,
    normalize_signal,
    resample,
    to_16bit,
    compute_rms,
    resample,
)
from clarity.utils.source_separation_support import get_device, separate_sources
from recipes.cad_icassp_2024.baseline.evaluate import (
    apply_gains,
    apply_ha,
    make_scene_listener_list,
    remix_stems,
    load_reference_stems,
)

logger = logging.getLogger(__name__)

/usr/local/lib/python3.10/dist-packages/recipes/cad_icassp_2024/baseline/evaluate.py:190: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="", config_name="config")


In [7]:
from recipes.cad_icassp_2024.baseline.enhance import (
    save_flac_signal,
    decompose_signal,
    process_remix_for_listener
)

/usr/local/lib/python3.10/dist-packages/recipes/cad_icassp_2024/baseline/enhance.py:182: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="", config_name="config")


Import the correct config file (hardcoded location)

In [8]:
from omegaconf import OmegaConf
config = OmegaConf.load('config.yaml')

Set input directory - This is where the reference and enhanced audio is loaded from

In [9]:
reference_folder = Path("Val_Ref_Tracks")
enhanced_mix_folder = Path("Val_enhanced_signals_openunmix_100") # change path to Val_enhanced_signals_openunmix_100 for openunmix eval

# load the audio files in provided path

In [10]:
enhanced_files = [] # initialize list of enhanced audio files
reference_files = [] # initialize list of reference files
for file in os.listdir(enhanced_mix_folder): # iterate over all files in the directory
    if file.endswith('.flac'): # if the file is an audio file
        enhanced_files.append(os.path.join(enhanced_mix_folder, file)) # add the file to the list of audio files
        reference_files.append(os.path.join(reference_folder, file.replace('.flac','_ref.flac'))) # add the corresponding ref file to the list of ref files

Load listener data

In [11]:
config.path.root = '/content/drive/MyDrive/Cadenza_Challenge/cad_icassp_2024/Validation'


In [12]:
# Load listener audiograms and songs
listener_dict = Listener.load_listener_dict(config.path.listeners_file)

with Path(config.path.gains_file).open("r", encoding="utf-8") as file:
    gains = json.load(file)

with Path(config.path.scenes_file).open("r", encoding="utf-8") as file:
    scenes = json.load(file)

with Path(config.path.scene_listeners_file).open("r", encoding="utf-8") as file:
    scenes_listeners = json.load(file)

with Path(config.path.music_file).open("r", encoding="utf-8") as file:
    songs = json.load(file)

Create a list of songs, listeners (audiogram) and head position (hrtf) to generate, or evaluate the mix - based on provided data

In [13]:
# Select a batch to process
scene_listener_pairs = make_scene_listener_list(
    scenes_listeners, config.evaluate.small_test
)

scene_listener_pairs = scene_listener_pairs[
    config.evaluate.batch :: config.evaluate.batch_size
]

Set up results CSV file (Already exists, so maybe change the name if you want to avoid rewriting)

In [34]:
scores_headers = [
    "scene",
    "song",
    "listener",
    "left_score",
    "right_score",
    "score",
]


results_file = ResultsFile(
            "Val_scores_HAAQI_openunmix.csv",
            header_columns=scores_headers,
)

# Run Eval
This will load both enhanced and reference mixtures, process them through HAAQI and save results to the CSV file

In [36]:
from recipes.cad_icassp_2024.baseline.evaluate import level_normalisation
previous_song = ""
num_scenes = len(scene_listener_pairs)
nrun = num_scenes # change to num_scenes for full dataset


for idx in range(100):
    # Extract track, listener, scene info from file name
    file_name = enhanced_files[idx]
    sname = file_name.split("/", 1)[1].split("_L",1)[0]
    lname = "L" +  file_name.split("/", 1)[1].split("_L",1)[1].split("_")[0]

    # Get the listener's audiogram
    listener = listener_dict[lname]
    scene = scenes[sname]
    song_name = f"{scene['music']}-{scene['head_loudspeaker_positions']}"

    print(sname)
    print(lname)

    # load audio files for HAAQI
    reference_mixture = read_signal(
            filename=reference_files[idx],
            sample_rate=config.sample_rate,
            allow_resample=True,
        )

    enhanced_mixture = read_signal(
            filename=enhanced_files[idx],
            sample_rate=config.sample_rate,
            allow_resample=True,
        )

    enhanced_mixture = level_normalisation(enhanced_mixture, reference_mixture, config.sample_rate)

    # Evaluate - compare with the generated mixes
    # Compute the scores
    left_score = compute_haaqi(
        processed_signal=resample(
            enhanced_mixture[:, 0],
            config.sample_rate,
            config.HAAQI_sample_rate,
        ),
        reference_signal=resample(
            reference_mixture[:, 0], config.sample_rate, config.HAAQI_sample_rate
        ),
        processed_sample_rate=config.HAAQI_sample_rate,
        reference_sample_rate=config.HAAQI_sample_rate,
        audiogram=listener.audiogram_left,
        equalisation=2,
        level1=65 - 20 * np.log10(compute_rms(reference_mixture[:, 0])),
    )

    right_score = compute_haaqi(
        processed_signal=resample(
            enhanced_mixture[:, 1],
            config.sample_rate,
            config.HAAQI_sample_rate,
        ),
        reference_signal=resample(
            reference_mixture[:, 1], config.sample_rate, config.HAAQI_sample_rate
        ),
        processed_sample_rate=config.HAAQI_sample_rate,
        reference_sample_rate=config.HAAQI_sample_rate,
        audiogram=listener.audiogram_right,
        equalisation=2,
        level1=65 - 20 * np.log10(compute_rms(reference_mixture[:, 1])),
    )

        #Save scores
    results_file.add_result(
        {
            "scene": sname,
            "song": song_name,
            "listener": listener.id,
            "left_score": left_score,
            "right_score": right_score,
            "score": float(np.mean([left_score, right_score])),
        }
    )

    print(right_score, left_score)


scene_50001
L5020
0.9895581026353326 0.9687147285639366
scene_50001
L5048
0.9766271983972724 0.9577958677673406
scene_50002
L5011
0.8284635786585051 0.8427877436083842
scene_50002
L5024
0.6826228687187442 0.8003539044427667
scene_50003
L5027
0.7842559841064657 0.7803888613281513
scene_50003
L5053
0.770275402588071 0.760859128198968
scene_50004
L5009
0.667895208262067 0.7396628537689767
scene_50004
L5013
0.656506221339491 0.6717366107335362
scene_50005
L5091
0.8923637187836828 0.8881943618098562
scene_50005
L5005
0.8990401043279207 0.8418892408132961
scene_50006
L5047
0.7112426020333524 0.7173436195823406
scene_50006
L5031
0.6487915869535694 0.5094641036145061
scene_50007
L5024
0.46972739630558435 0.5985368191928933
scene_50007
L5018
0.5304869097754585 0.6219010082662457
scene_50008
L5007
0.6239216955388562 0.7073628015972064
scene_50008
L5086
0.8086260969680856 0.8052885510008825
scene_50009
L5066
0.8378098826704223 0.8533703111920803
scene_50009
L5018
0.8137159347786957 0.837896606417

0.162 0.8133482388153274
scene_50020
L5014
0.7374317831017563 0.7476014149182897
scene_50020
L5039
0.7900655016670781 0.7775179361677821
scene_50021
L5063
0.8562247137760309 0.8695588780932444
scene_50021
L5064
0.8836297987578654 0.8741137719609486
scene_50022
L5079
0.4458889634332541 0.4987298011368968
scene_50022
L5078
0.4505930027156199 0.48564718924271544
scene_50023
L5042
0.7932405927955806 0.810396686767309
scene_50023
L5005
0.7997581895330494 0.8366371414415008
scene_50024
L5027
0.7428955803353996 0.7808205663593747
scene_50024
L5085
0.6401962449858177 0.7237978555042635
scene_50025
L5063
0.6599455473072176 0.5945785421513206
scene_50025
L5079
0.6523893236407219 0.6640750435552685
scene_50026
L5069
0.5730450855912668 0.6055906418618208
scene_50026
L5041
0.7125605592401169 0.6091013547878859
scene_50027
L5069
0.6138946661447444 0.6680819265033715
scene_50027
L5009
0.5393832650254757 0.8042304040585354
scene_50028
L5076
0.9084703203346338 0.901939111590597
scene_50028
L5019
0.8696

0.162 0.449274294410564
scene_50042
L5023
0.489778930715573 0.5140731389579962
scene_50043
L5079
0.7108982039051854 0.7155989453493661
scene_50043
L5062
0.7105476940290488 0.7318558354511298
scene_50044
L5086
0.8164358808573788 0.8259680942055044
scene_50044
L5071
0.8292523908705591 0.8304424004539108
scene_50045
L5039
0.3966870824250778 0.44140960890749326
scene_50045
L5045
0.40567574479654656 0.42067547080452883
scene_50046
L5999
0.5759187128712686 0.5574280587633091
scene_50046
L5019
0.5995733910838895 0.6445451914986072
scene_50047
L5043
0.5657824268548257 0.6365581402512765
scene_50047
L5078
0.5554299735560764 0.5838403752578042
scene_50048
L5014
0.7365787317509991 0.7453847133419027
scene_50048
L5005
0.8199220002066376 0.805373400780507
scene_50049
L5057
0.6240643162655668 0.6026235152392436
scene_50049
L5020
0.6587875678189653 0.7312345411603614
scene_50050
L5029
0.7162732452236653 0.6489557113524255
scene_50050
L5076
0.8747065572321866 0.8549499726381484


In [25]:
enhanced_mixture = enhanced_mixture/np.max(np.abs(enhanced_mixture),0)
print(np.max(np.abs(enhanced_mixture[:,1])))

1.0
